# Defining the aoi

In [ ]:
with open(f"{root_path}/core/geo-countries/archive/countries.geojson", "rb") as f:
    all_countries_geojson = json.loads(f.read())


for i in all_countries_geojson['features']:
    if i['properties']['ADMIN'] == "France":
        france_geo = i


france_gdf = gpd.GeoDataFrame.from_features([france_geo]).explode()

france_gdf[france_gdf.area==max(france_gdf.area)].plot()

aoi = france_gdf[france_gdf.area==max(france_gdf.area)]

aoi_geojson = json.loads(aoi.to_json())

xmin, ymin, xmax, ymax = aoi.total_bounds
xmin, ymin, xmax, ymax = (4, 42, 7, 47)  # hardcoding since concattenating 1000s of ecostress files with different overlaps hangs

# Global rivers dataset

In [ ]:
rivers_df = gpd.read_file(Path(root_path, "europe_rivers/eu_river.shp"))

rivers_df['R_ID'] = rivers_df['R_ID'].apply(int).apply(str)

france_rivers_df = rivers_df.cx[xmin:xmax, ymin:ymax]

### Check to make sure they overlay, geopandas has poor support for linestrings so we subset by the bounding box

In [ ]:
base = aoi.plot(color="grey", edgecolor="black")

france_rivers_df.plot(ax=base, color="blue")

In [ ]:
import pandas as pd
import numpy as np
import xarray as xa
from pathlib import Path
import src.data.ecostress_io as eio
import rioxarray
import sys
import geopandas as gpd
import json

root_path = Path("/scratch/rave/")

qa_path = Path(root_path, "rhone-ecostress-data", "ECO3ANCQA")
et_path = Path(root_path, "rhone-ecostress-data", "ECO3ETPTJPL")
esi_path = Path(root_path, "rhone-ecostress-data", "ECO4ESIPTJPL")

tif_qa_paths, csv_qa_paths, xml_qa_paths = eio.separate_extensions(qa_path)

tif_etdaily_paths, csv_et_paths, xml_et_paths = eio.separate_extensions(et_path, "*ETdaily*.tif")

et = eio.read_mask_ecostress_scene(tif_etdaily_paths[0])

Can use below to filter out all xarray data arrays that do not at least partially intersect with an aoi boundary. Then hopefully concatenating can happen.

In [ ]:
try:
    et_clipped = et.rio.clip_box(
        minx=xmin,
        miny=ymin,
        maxx=xmax,
        maxy=ymax,
    )
except rioxarray.exceptions.NoDataInBounds:
    print("The whole scene falls outside the aoi bounds, skipping")

In [ ]:

def read_scenes(tif_paths):
    """
    Reads in multiple Landsat surface reflectance scenes given a regex pattern.
    Args:
        path (str): Path of form "../*".
    Returns:
        bool: Returns an xarray data array with dimensions for x, y, band, and time.
    """
    scenes = [eio.read_mask_ecostress_scene(path) for path in tif_paths]
    return xa.merge(scenes)

In [ ]:
all_et_daily = read_scenes(tif_etdaily_paths)

all_et_daily.nbytes / 1e9

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib inline

ETcolors = ["#f6e8c3", "#d8b365", "#99974a", "#53792d", "#6bdfd2", "#1839c5"]
ETcmap = LinearSegmentedColormap.from_list("ET", ETcolors)
date_utc = pd.to_datetime(et['date'].values)
layer_type = et.attrs['filename'].split("_")[-3]
title = 'ECO3ETPTJPL Evapotranspiration'

fig = plt.figure(figsize=(9.7,7.6))                                                       # Set the figure size (x,y)
fig.suptitle(f'{title} ({layer_type}) \n at {date_utc}', fontsize=22)  # Add title for the plots
plt.axis('off')                                                                           # Remove axes from plot
im = plt.imshow(et.sel(band=1), cmap=ETcmap);                                                        # Plot array using colormap
# plt.scatter(Tcol, Trow, color="black", marker='x')                                        # Plot tower location
# Add a colormap legend
plt.colorbar(im, orientation='horizontal', fraction=0.05, pad=0.004, label=f"ET ({et.attrs['units']})", shrink=0.6).outline.set_visible(True)

In [ ]:
xa.open_rasterio(tif_qa_paths[0], parse_coordinates=True)

## Code graveyard

In [ ]:
def mask_NA_values():
    """
    Daily ET products have both nan values from where there are clouds 
    and -1e+13 for where the ecostress swath was clipped during the ordering process
    """
    masked_et = np.ma.masked_where(et.sel(band=1) == np.nan, et.sel(band=1))
    masked_et = np.ma.masked_where(masked_et == -1e+13, masked_et)